In [1]:
import random
import copy
import time

# PARTE 1: Estrutura da tarefa

class Tarefa:
    # Classe pra representar uma tarefa com id, tempo e data de entrega
    def __init__(self, id, tempo_processamento, data_entrega):
        self.id = id
        self.tempo_processamento = tempo_processamento
        self.data_entrega = data_entrega

    def __repr__(self):
        # Só pra imprimir a tarefa de forma mais legível
        return f"T{self.id}(p={self.tempo_processamento}, d={self.data_entrega})"


# PARTE 2: Função pra calcular o atraso total

def calcular_atraso_total(sequencia_tarefas):
    # Calcula o atraso somado de uma lista de tarefas na ordem que elas aparecem
    tempo_conclusao_atual = 0
    atraso_total = 0

    for tarefa in sequencia_tarefas:
        tempo_conclusao_atual += tarefa.tempo_processamento
        atraso_tarefa = max(0, tempo_conclusao_atual - tarefa.data_entrega)
        atraso_total += atraso_tarefa

    return atraso_total


# PARTE 3: Geração de vizinhos (trocas simples pra buscar solução melhor)

def gerar_vizinho_swap(sequencia):
    # Troca duas tarefas de lugar na sequência (aleatoriamente)
    n = len(sequencia)
    if n < 2:
        return list(sequencia)

    vizinho = list(sequencia)
    idx1, idx2 = random.sample(range(n), 2)
    vizinho[idx1], vizinho[idx2] = vizinho[idx2], vizinho[idx1]
    return vizinho

def gerar_vizinho_insertion(sequencia):
    # Remove uma tarefa e coloca em outro lugar da lista
    n = len(sequencia)
    if n < 2:
        return list(sequencia)

    vizinho = list(sequencia)
    idx_removida = random.randint(0, n - 1)
    tarefa_removida = vizinho.pop(idx_removida)
    idx_inserir = random.randint(0, n)
    vizinho.insert(idx_inserir, tarefa_removida)
    return vizinho


# PARTE 4: Meta-heurística básica (busca local)

def metaheuristica_generica_busca_local(tarefas, max_iteracoes=1000, tipo_movimento='swap'):
    # Busca uma boa ordem de execução das tarefas testando vizinhos
    sequencia_atual = random.sample(tarefas, len(tarefas)) # Começa com uma ordem aleatória
    melhor_sequencia = list(sequencia_atual)
    melhor_atraso = calcular_atraso_total(melhor_sequencia)

    print(f"Início da Busca: Atraso Inicial = {melhor_atraso}")
    print(f"Sequência Inicial: {[t.id for t in melhor_sequencia]}")

    for iteracao in range(max_iteracoes):
        if tipo_movimento == 'swap':
            vizinho = gerar_vizinho_swap(sequencia_atual)
        elif tipo_movimento == 'insert':
            vizinho = gerar_vizinho_insertion(sequencia_atual)
        else:
            raise ValueError("Tipo de movimento inválido")

        atraso_vizinho = calcular_atraso_total(vizinho)

        if atraso_vizinho < calcular_atraso_total(sequencia_atual):
            sequencia_atual = vizinho
            if atraso_vizinho < melhor_atraso:
                melhor_atraso = atraso_vizinho
                melhor_sequencia = list(sequencia_atual)

    print(f"\nFim da Busca: Melhor Atraso = {melhor_atraso}")
    print(f"Melhor Sequência: {[t.id for t in melhor_sequencia]}")

    return melhor_sequencia, melhor_atraso


# PARTE 5: Testes com dados fictícios

if __name__ == "__main__":
    print("--- GERANDO TAREFAS DE TESTE ---")
    tarefas_para_teste = [
        Tarefa(id=1, tempo_processamento=10, data_entrega=20),
        Tarefa(id=2, tempo_processamento=5, data_entrega=12),
        Tarefa(id=3, tempo_processamento=8, data_entrega=18),
        Tarefa(id=4, tempo_processamento=7, data_entrega=15),
        Tarefa(id=5, tempo_processamento=12, data_entrega=30),
        Tarefa(id=6, tempo_processamento=6, data_entrega=14),
        Tarefa(id=7, tempo_processamento=9, data_entrega=25),
        Tarefa(id=8, tempo_processamento=4, data_entrega=11)
    ]
    for t in tarefas_para_teste:
        print(t)
    print("-" * 30)

    print("--- TESTANDO A FUNÇÃO DE ATRASO COM SEQUÊNCIA ALEATÓRIA ---")
    sequencia_inicial_teste = random.sample(tarefas_para_teste, len(tarefas_para_teste))
    print(f"Sequência: {[t.id for t in sequencia_inicial_teste]}")
    atraso_inicial = calcular_atraso_total(sequencia_inicial_teste)
    print(f"Atraso Total: {atraso_inicial}\n")
    print("-" * 30)

    print("--- RESULTADOS DA BUSCA LOCAL ---")

    print("\nExecutando com SWAP:")
    start_time_swap = time.time()
    melhor_seq_swap, melhor_atraso_swap = metaheuristica_generica_busca_local(
        tarefas_para_teste, max_iteracoes=5000, tipo_movimento='swap'
    )
    end_time_swap = time.time()
    print(f"\nMelhor Ordem (SWAP): {[t.id for t in melhor_seq_swap]}")
    print(f"Melhor Atraso (SWAP): {melhor_atraso_swap}")
    print(f"Tempo: {end_time_swap - start_time_swap:.4f} segundos")
    print("-" * 30)

    print("\nExecutando com INSERTION:")
    start_time_insert = time.time()
    melhor_seq_insert, melhor_atraso_insert = metaheuristica_generica_busca_local(
        tarefas_para_teste, max_iteracoes=5000, tipo_movimento='insert'
    )
    end_time_insert = time.time()
    print(f"\nMelhor Ordem (INSERTION): {[t.id for t in melhor_seq_insert]}")
    print(f"Melhor Atraso (INSERTION): {melhor_atraso_insert}")
    print(f"Tempo: {end_time_insert - start_time_insert:.4f} segundos")
    print("-" * 30)

    # Heurística simples EDD (Earliest Due Date)
    print("\n--- TESTE COM HEURÍSTICA CONSTRUTIVA (EDD) ---")
    seq_edd = sorted(tarefas_para_teste, key=lambda t: t.data_entrega)
    atraso_edd = calcular_atraso_total(seq_edd)
    print(f"Ordem EDD: {[t.id for t in seq_edd]}")
    print(f"Atraso Total EDD: {atraso_edd}")
    print("-" * 30)


--- GERANDO TAREFAS DE TESTE ---
T1(p=10, d=20)
T2(p=5, d=12)
T3(p=8, d=18)
T4(p=7, d=15)
T5(p=12, d=30)
T6(p=6, d=14)
T7(p=9, d=25)
T8(p=4, d=11)
------------------------------
--- TESTANDO A FUNÇÃO DE ATRASO COM SEQUÊNCIA ALEATÓRIA ---
Sequência: [3, 1, 6, 8, 4, 2, 5, 7]
Atraso Total: 133

------------------------------
--- RESULTADOS DA BUSCA LOCAL ---

Executando com SWAP:
Início da Busca: Atraso Inicial = 133
Sequência Inicial: [2, 7, 6, 1, 8, 3, 5, 4]

Fim da Busca: Melhor Atraso = 94
Melhor Sequência: [8, 2, 6, 4, 3, 7, 1, 5]

Melhor Ordem (SWAP): [8, 2, 6, 4, 3, 7, 1, 5]
Melhor Atraso (SWAP): 94
Tempo: 0.0535 segundos
------------------------------

Executando com INSERTION:
Início da Busca: Atraso Inicial = 177
Sequência Inicial: [5, 1, 3, 6, 8, 7, 2, 4]

Fim da Busca: Melhor Atraso = 94
Melhor Sequência: [8, 2, 6, 4, 3, 7, 1, 5]

Melhor Ordem (INSERTION): [8, 2, 6, 4, 3, 7, 1, 5]
Melhor Atraso (INSERTION): 94
Tempo: 0.0486 segundos
------------------------------

--- TESTE CO